# A2.2 Part-of-Speech Tagging using HMM  
**Author:** Zak Hussain  
**Weeks:** week 3 & 4

**Purpose**  
The goal for this section is to build a POS tagger using HMM as explained the section 8.4 reading: https://web.stanford.edu/~jurafsky/slp3/8.pdf.  

The Training data used comes from the brown corpus. Each file contains sentences of tokenized words followed by POS tags and where each line contains a sentence. Here is the link to the dataset: https://www.dropbox.com/s/m22lg4kucvpv815/brown.zip?dl=0

The test set is also from the brown corpus. Each file contains tokenized words, but no tages. Here is the link to the dataset: https://www.dropbox.com/s/bg0cfdd2igjrmyl/Test_File.txt?dl=0

Implement a POS tagger using a bigram HMM. Given an observed sequence of n words w1, w2, ..wn, choose the most probable sequence of POS tags  t1, t2,...tn. 

**NOTE:** during training, choose a *frequency threshold* (e.g., 5) and trat any words that have freq below this as unknown. Use observations to select your observations. 

**Tasks:**
* 2.1 - Frequency Counts 
* 2.2 - Transition Probabilities 
* 2.3 - Emission Probabilities 
* 2.4 - Viterbi Algorithm 

### 2.1 - Frequency Counts:   
Obtain Frequency Counts from the training set. Gather Tag-word pair counts, Tag-unigram counts, Tag-bigram counts.

**Step 0.a.**  
create list that contains tokenized sentences across the entire corpus.

In [1]:
# get the training data
from nltk.util import ngrams 
import os 

# initialize the path.  
training_data_path = '../../Week3&4/Data_Part2/Training/brown/' 

# init a list to contain lists, where each list is a tokenized sentence. 
corpus_sent = [] 
 
# for each file, tokenize the sentence, and 
for filename in os.listdir(training_data_path):
    with open(training_data_path + filename, 'r') as txt_file: 
        # drop the newline character and don't add empty lists to the corpus list.
        for line in txt_file:
            temp = line.strip('\n') 
            if temp != '':
                corpus_sent.append(line.split())
print('This is what the first two elements in the list looks like--each list is a tokenized sentence.')
corpus_sent[0:2]

This is what the first two elements in the list looks like--each list is a tokenized sentence.


[['The/at',
  'Fulton/np-tl',
  'County/nn-tl',
  'Grand/jj-tl',
  'Jury/nn-tl',
  'said/vbd',
  'Friday/nr',
  'an/at',
  'investigation/nn',
  'of/in',
  "Atlanta's/np$",
  'recent/jj',
  'primary/nn',
  'election/nn',
  'produced/vbd',
  '``/``',
  'no/at',
  'evidence/nn',
  "''/''",
  'that/cs',
  'any/dti',
  'irregularities/nns',
  'took/vbd',
  'place/nn',
  './.'],
 ['The/at',
  'jury/nn',
  'further/rbr',
  'said/vbd',
  'in/in',
  'term-end/nn',
  'presentments/nns',
  'that/cs',
  'the/at',
  'City/nn-tl',
  'Executive/jj-tl',
  'Committee/nn-tl',
  ',/,',
  'which/wdt',
  'had/hvd',
  'over-all/jj',
  'charge/nn',
  'of/in',
  'the/at',
  'election/nn',
  ',/,',
  '``/``',
  'deserves/vbz',
  'the/at',
  'praise/nn',
  'and/cc',
  'thanks/nns',
  'of/in',
  'the/at',
  'City/nn-tl',
  'of/in-tl',
  'Atlanta/np-tl',
  "''/''",
  'for/in',
  'the/at',
  'manner/nn',
  'in/in',
  'which/wdt',
  'the/at',
  'election/nn',
  'was/bedz',
  'conducted/vbn',
  './.']]

**Step 0.b.**   
Create a collection for the word sequences and tag sequences. for all sequences, the start and stop tokens are incorporated at the start and end of all sequences, in both collections. This also means, that I am treating the start and stop tokens as two unique tags. I am choosing to do this arbitrarily, but it also keeps the indexes of each tag sequence in line with its associated word sequence.

In [2]:
# create two seperate lists, one for words, and the other for tags. 
word_sequences = [] 
tag_sequences = [] 

for tokenized_sentences in corpus_sent:
    word_sequence = ['<s>']
    tag_sequence = ['<s>']
    
    for tagged_word in tokenized_sentences:
        # split the tagged word at the '/' 
        split_tagged_word = tagged_word.split('/')
        
        # edge case, to handle this exact sequence of chars: ''/''
        if split_tagged_word[0] == '': 
            word_sequence.append('')
            tag_sequence.append('')
            
        # edge case, where a word in the corpus is not tagged: 'ca01' had occured in the corpus with no tag.
        elif len(split_tagged_word) < 2: 
            continue 
        
        # else append the word and tag to the associated collection
        else: 
            word_sequence.append(split_tagged_word[0])
            tag_sequence.append(split_tagged_word[1])
            
    # add the (sequences with a stop token) to their associated collections.
    word_sequence.append('</s>')
    tag_sequence.append('</s>')
    word_sequences.append(word_sequence) 
    tag_sequences.append(tag_sequence)

print('shown is 1 sequence of words and its associated tag sequence (be wary of the commas):\n')
print(word_sequences[1]) 
print()
print(tag_sequences[1])
print('==========================================================')

print("notice the size of the word_sequences are equal to that of the tag_sequences.")
print('length of the word sequences', len(word_sequences))
print('length of the tag sequences', len(tag_sequences))

shown is 1 sequence of words and its associated tag sequence (be wary of the commas):

['<s>', 'The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.', '</s>']

['<s>', 'at', 'nn', 'rbr', 'vbd', 'in', 'nn', 'nns', 'cs', 'at', 'nn-tl', 'jj-tl', 'nn-tl', ',', 'wdt', 'hvd', 'jj', 'nn', 'in', 'at', 'nn', ',', '``', 'vbz', 'at', 'nn', 'cc', 'nns', 'in', 'at', 'nn-tl', 'in-tl', 'np-tl', "''", 'in', 'at', 'nn', 'in', 'wdt', 'at', 'nn', 'bedz', 'vbn', '.', '</s>']
notice the size of the word_sequences are equal to that of the tag_sequences.
length of the word sequences 57841
length of the tag sequences 57841


**Step 1.**  
Generate the Tag-word pair counts: denoted C (t_j, w_j):

In [3]:
from collections import defaultdict

# create dictionary collection of {tag:{word:Freq}}
tag_word_pair_counts = defaultdict(lambda: defaultdict(lambda:0)) 

# I will treat start and stop tokens tags. Also, this is an O(n) iteration, where n is the total number of words in the corpus.
for i in range(len(word_sequences)): 
    for j in range(len(word_sequences[i])): 
        tag_word_pair_counts[tag_sequences[i][j]][word_sequences[i][j]] += 1

print('this is an example of the word-token frequencies for a specific tag "at":')
tag_word_pair_counts['at']

this is an example of the word-token frequencies for a specific tag "at":


defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
            {'The': 6725,
             'an': 3518,
             'no': 1575,
             'the': 62288,
             'a': 21824,
             'A': 1119,
             'every': 434,
             'An': 188,
             'Every': 56,
             'No': 230,
             "th'": 1,
             "ever'": 1})

**Step 2.**   
Generate the Tag-unique counts: denoted C(t_i):

In [4]:
tag_unique_counts = defaultdict(lambda: 0) 

for tag_sequence in tag_sequences: 
    for tag in tag_sequence: 
        tag_unique_counts[tag] += 1
        
print('Here is an example of a tag frequency value for the tag "nn"')
print('There are {} occurances of the tag "nn" in tag_unique counts.'.format(tag_unique_counts['nn']))
print()
print('There are a total of {} unique tags in this dictionary.'.format(len(tag_unique_counts.keys())))

Here is an example of a tag frequency value for the tag "nn"
There are 152393 occurances of the tag "nn" in tag_unique counts.

There are a total of 525 unique tags in this dictionary.


**Step 3.**  
Generate the Tag-bigram counts: denoted C(T_(i-1), T_i): 

In [5]:
# generate the bigrams for every tag_sequences in all tag_sequences. 
bigram_sequences = [list(ngrams(tag_sequence, 2)) for tag_sequence in tag_sequences]
print(bigram_sequences[0])

[('<s>', 'at'), ('at', 'np-tl'), ('np-tl', 'nn-tl'), ('nn-tl', 'jj-tl'), ('jj-tl', 'nn-tl'), ('nn-tl', 'vbd'), ('vbd', 'nr'), ('nr', 'at'), ('at', 'nn'), ('nn', 'in'), ('in', 'np$'), ('np$', 'jj'), ('jj', 'nn'), ('nn', 'nn'), ('nn', 'vbd'), ('vbd', '``'), ('``', 'at'), ('at', 'nn'), ('nn', "''"), ("''", 'cs'), ('cs', 'dti'), ('dti', 'nns'), ('nns', 'vbd'), ('vbd', 'nn'), ('nn', '.'), ('.', '</s>')]


In [6]:
# check to ensure that the bigram_sequences[0] correctly corresponds to tag_sequences[0].
print('as we can see, every element in the bigram above is correctly generated based on an existing\n' +
      'tag sequence at a given index in the tag_sequences collection.')
print()
print(tag_sequences[0])

as we can see, every element in the bigram above is correctly generated based on an existing
tag sequence at a given index in the tag_sequences collection.

['<s>', 'at', 'np-tl', 'nn-tl', 'jj-tl', 'nn-tl', 'vbd', 'nr', 'at', 'nn', 'in', 'np$', 'jj', 'nn', 'nn', 'vbd', '``', 'at', 'nn', "''", 'cs', 'dti', 'nns', 'vbd', 'nn', '.', '</s>']


In [7]:
# generate the tag_bigram_counts.
tag_bigram_counts = defaultdict(lambda: defaultdict(lambda : 0))

for lst_of_bigrams in bigram_sequences: 
    for bigram in lst_of_bigrams: 
        tag_bigram_counts[bigram[0]][bigram[1]] += 1
        
print('Here is an example of the tag_bigram_counts for the tags that have "at" as their preceding tag in a sequence:') 
print()
print(tag_bigram_counts['at'])

Here is an example of the tag_bigram_counts for the tags that have "at" as their preceding tag in a sequence:

defaultdict(<function <lambda>.<locals>.<lambda> at 0x000001C21FDDF2F0>, {'np-tl': 809, 'nn': 48368, 'nn-tl': 2564, 'np': 2228, 'jj': 19480, 'jjt': 675, 'ap': 3007, 'nns': 7214, 'nn$': 907, 'vbg': 1568, 'cd': 977, 'jjs': 206, 'vbn': 1468, 'jj-tl': 1412, 'nps': 588, 'od': 1251, '``': 620, 'nns$': 97, 'rb': 350, 'ql': 1377, 'jjs-tl': 2, 'nn$-tl': 162, 'jjr': 630, 'vbn-tl': 390, 'nr-tl': 208, 'nns-tl': 284, 'fw-in': 7, 'abn': 42, 'nr': 218, 'nps$': 30, 'pn': 149, 'nns$-tl': 28, '*': 4, '2': 3, '</s>': 14, 'np$': 62, "'": 24, 'vbg-tl': 34, 'od-tl': 98, 'jjr-tl': 3, '2-story': 1, 'fw-nn-tl': 52, 'rb-tl': 1, 'cd-tl': 29, 'fw-jj-tl': 8, 'nr$-tl': 8, 'fw-nn': 76, '2-inch': 3, 'rbt': 11, '(': 15, "''": 7, 'cc': 3, 'vb': 16, 'rb-nc': 1, 'vbz': 1, 'rp': 1, 'np$-tl': 17, 'vbd': 2, ',': 10, '--': 4, 'pn$': 1, 'rbr': 40, 'nn+hvz': 3, 'fw-in-tl': 4, 'in': 3, 'nn+bez': 13, "2''": 1, "4''": 2,

### Step 2.2 - Transition Probabilities:  
Compute the transition probability of a tag given its previous tag. 

\begin{equation*}
P(t_i | t_{i-1}) = \frac{C(t_{i-1}, t_i)}{C(t_{i-1})}
\end{equation*}

In [8]:
# Purpose: compute the transition probabilities for a tag given a historical tag occured. 
def get_transition_prob(tag: str, historical_tag: str) -> float:
    # handle edge case, where an arguement is ''.
    if tag == '': 
        tag = "''"
    if historical_tag == '': 
        historical_tag = "''"
    return float(tag_bigram_counts[historical_tag][tag] / tag_unique_counts[historical_tag])  

In [9]:
# create a precomputed table A, as a dictionary that is stores the transition probs. 
# the transition prob is accessed by A[tag_(i-1)][tag_i]
A = defaultdict(lambda: defaultdict(lambda: 0)) 

for historical_tag in tag_bigram_counts.keys(): 
    for tag in tag_bigram_counts[historical_tag].keys(): 
        A[historical_tag][tag] = get_transition_prob(tag, historical_tag) 

### Step 2.3 - Emission Probabilies:   
Compute the emission probability of a word given a tag.  

\begin{equation*} 
P(w_i|t_i) = \frac{C(t_i, w_i}{C(t_i)}
\end{equation*}

In [10]:
def get_emission_prob(word:str, tag:str) -> float: 
    # handle edge case, where a tag is ''.
    if tag == '': 
        tag = "''"
    return float(tag_word_pair_counts[tag][word] / tag_unique_counts[tag])

In [11]:
# create a precomputed table B, as a dictionary that stores the emission probs. 
# the emission prob is accesed by B[tag][word]
B = defaultdict(lambda: defaultdict(lambda: 0)) 

for tag in tag_word_pair_counts.keys(): 
    for word in tag_word_pair_counts[tag].keys(): 
        B[tag][word] = get_emission_prob(word, tag)

### Step 2.4 - Viterbi Algorithm:  
for each word in the test set, derive the most probable POS tag sequence using the veterbi algorithm. 

In [12]:
# read in test file, and seperate out the sentences into token sequences.

import re 
test_file = '../../Week3&4/Data_Part2/Test/Test_File.txt'
test_token_sequences = [] 

# identifier for the start and end of a sentence
regex_start = re.compile(r'< sentence ID =[\d]')
regex_end = re.compile(r'<EOS>')

with open(test_file, 'r') as txt_file: 
    line = (txt_file.readline()).strip('\n')
    line = line.strip('\n')
    
    while (line): 
        
        # found the start of a sequence
        if regex_start.match(line):
            sequence = ['<s>']
            line = (txt_file.readline()).strip('\n')
            while (not regex_end.match(line)): 
                sequence.append(line)
                line = (txt_file.readline()).strip('\n')
            # add the stop token to the sequence 
            sequence.append('</s>') 
            # add the sequence to all sequences
            test_token_sequences.append(sequence) 
            
        line = (txt_file.readline()).strip('\n') 

print('Here is an example of how the first test sequence looks:')
test_token_sequences[0]

Here is an example of how the first test sequence looks:


['<s>',
 'We',
 'have',
 'learned',
 'much',
 'about',
 'interstellar',
 'drives',
 'since',
 'a',
 'hundred',
 'years',
 'ago',
 ';',
 ';',
 'that',
 'is',
 'all',
 'I',
 'can',
 'tell',
 'you',
 'about',
 'them',
 '.',
 '</s>']

In [13]:
# generate the initial probability distribution. 
pi = defaultdict(lambda:0.00) 

# save the total sum of tag frequencies to variable. 
total_tag_freqs = sum(tag_unique_counts.values())

# generate the pi distribution
for tag in tag_unique_counts: 
    pi[tag] = tag_unique_counts[tag]/total_tag_freqs
    
# check that the sum of the values is equal to 1. 
print("the sum of the values in the initial probability distribution is: ", round(sum(pi.values()), 2))

the sum of the values in the initial probability distribution is:  1.0


In [14]:
# helper functions for the viterbi algorithm 
import math 

# purpose: finds which state is most frequence
def get_most_frequent_state(pointer, viterbi, competing_state_idx, column) -> tuple:
    # get the frequency of occurances for the state the pointer currently holds. 
    current_state_freq = tag_unique_counts[states[pointer[0]]]
    
    # get the freq of occ. for the competing state
    competing_state_freq = tag_unique_counts[states[competing_state_idx]]
    
    # compare the frequencies, and return a pointer (tuple) to the most frequent state. 
    if current_state_freq > competing_state_freq: 
        return pointer
    return (competing_state_idx, column, viterbi[competing_state_idx][column])

# purpose: gets the max value from the previous cell
# the larger values result in smaller negatives. 
def find_max_column_state(viterbi:dict, column:int, num_rows:int,  backpointers: list): 
    # create a pointer to store the row_idx, colum_idx, and value
    pointer = (0, column, viterbi[0][column]) 
    
    for state_index in range(1, num_rows): 
        # if the current max is smaller than the new viterbi value, update the pointer.
        if pointer[2] > viterbi[state_index][column]: 
            pointer = (state_index, column, viterbi[state_index][column]) 
        
        # I make an assumption, that if two values are equal, I will use the more frequent of the two states.
        elif pointer[2] == viterbi[state_index][column]:
            pointer = get_most_frequent_state(pointer, viterbi, state_index, column)  
            
    # the max state becomes a part of our path. 
    backpointers.append(pointer) 


The original Implementation of viterbi, resulted in underflow resulting from the following equation used to compute the most probable tage sequence from a bigram tagger: 

\begin{equation*}
    {\hat t}{_l^n} = argmax_{t{_l^n}}\sideset{}{^n}\prod_{i=1}P(w_i|t_i)P(t_i | t_{i-1})
\end{equation*}  

To handle this problem, I transformed my probabilities to log space, which is results in the following: 

\begin{equation*}
    {\hat t}{_l^n} = argmax_{{t}{_l^n}}\sideset{}{^n}\sum_{i=1}logProb(w_i|t_i) + logProb(t_i | t_{i-1})
\end{equation*}

Thus the calculation for a cell from the viterbi lattice went from being in the multiplicative domain: 

\begin{equation*}
    v_t(j) = max_{i=1}v_{t-1}(i)a_{ij}b_{j}(o_t)
\end{equation*} 

to the additive domain as follows: 

\begin{equation*}
    v_t(j) = max_{i=1}v_{t-1}(i) + a_{ij} + b_{j}(o_t)
\end{equation*} 

In [15]:
# define the unique states globally. 
states = list(tag_unique_counts.keys())

# recall (defined above in 2.2 and 2.3) we are using A as the transition matrix, 
# and B as the emission matrix. 
     
# define the viterbi algorithm using logprobs: 
def viterbi(word_sequence:list) -> list: 
    # init viterbi as a dict of dicts, (its cleaner than using lists,) 
    # viterbi[row_index][column_index], where the keys are int values. 
    vit_lattice = defaultdict(lambda: defaultdict(lambda:0.00))
    
    # init the backpointer sequence as a list containing tuple(state, observation, value)  
    backpointers = [] 
    
    # init the stopping bounds of the matrix 
    num_states = len(states)   
    num_observations = len(word_sequence) 
    
    # generate the first column. 
    for state_index in range(num_states): 
        
        # precompute the initial transition and emission prob.
        pi_state = pi[states[state_index]]
        emission_prob = B[states[state_index]][word_sequence[0]]
        
        # if the pi or emission state is 0, just set the value to 0 becuase it can't be mapped to log space, 
        # and in the multiplicitave space the lattice value would have resulted in 0 as well. 
        if pi_state == 0 or emission_prob == 0: 
            vit_lattice[state_index][0] = 0 
        else: 
            vit_lattice[state_index][0] = math.log(pi_state) + math.log(emission_prob) 
        
    # use the helper functions to update the backpointer, i.e., add the 1st element in the tag sequence  
    find_max_column_state(vit_lattice, 0, num_states, backpointers)
    
    # generate the remaining columns in the tag sequence
    for column_index in range(1, num_observations): 
        for state_index in range(num_states): 
            
            # get the previous columns max value, and the associated state. 
            prev_val = backpointers[column_index - 1][2] 
            prev_tag = states[backpointers[column_index - 1][0]]
            
            # identify if any of these values are 0 before converting to log space
            transition_prob = A[prev_tag][states[state_index]]
            emission_prob = B[states[state_index]][word_sequence[column_index]]
            if transition_prob == 0 or emission_prob == 0: 
                vit_lattice[state_index][column_index] = 0 
            else: 
                # compute the new column values = prev_val + A[t_i][t_i-1] + B[tag][word]
                vit_lattice[state_index][column_index] = (prev_val + 
                                                         math.log(transition_prob) + 
                                                         math.log(emission_prob))
            
        # use the helper functions to update the backpointer, i.e., add the next element in the tag sequence  
        find_max_column_state(vit_lattice, column_index, num_states, backpointers)            
    
    return backpointers 

In [16]:
# use the global 'states' variable, and local test_token_sequence[i] to generate a list of the (tag:word) sequence
def generate_tag_sequence(backpointers: list, word_sequence: list) -> list: 
    tag_seq = [] 
    for pointer in backpointers:
        state = states[pointer[0]]
        observation = word_sequence[pointer[1]] 
        tag_seq.append((observation, state)) 
    return tag_seq

In [17]:
# test the viterbi algo on one sequence, and generate the (word, tag) outcome. 
l = viterbi(test_token_sequences[0])
generate_tag_sequence(l, test_token_sequences[0])

[('<s>', '<s>'),
 ('We', 'ppss-hl'),
 ('have', 'hv-hl'),
 ('learned', 'nn'),
 ('much', 'ap'),
 ('about', 'rp'),
 ('interstellar', 'jj'),
 ('drives', 'vbz'),
 ('since', 'rb'),
 ('a', 'at-tl'),
 ('hundred', 'cd'),
 ('years', 'nns'),
 ('ago', 'rb'),
 (';', '.'),
 (';', '.'),
 ('that', 'nn'),
 ('is', 'nil'),
 ('all', 'nn'),
 ('I', 'nn-tl'),
 ('can', 'vb'),
 ('tell', 'vb-nc'),
 ('you', 'ppo'),
 ('about', 'rp'),
 ('them', 'dts'),
 ('.', '.'),
 ('</s>', '</s>')]

Execute the viterbi algorithm on the test set, and generate a collection containing the (word,tag) sequences. 

In [18]:
# create a collection of (word, tag) tuple sequences 
resulting_sequences = [] 

# generate the (word, tag) sequences by using 
# viterbi and mapping the backpointers to the word and tag. 
for word_sequence in test_token_sequences: 
    backpointers = viterbi(word_sequence) 
    resulting_sequences.append(generate_tag_sequence(backpointers, word_sequence))
    

In [19]:
# write to the results to a file: 
file = open('HMM_predictions.txt', "w") 
for sequence_num, tupls in enumerate(resulting_sequences, start=1): 
    # start of a sentence sequence. 
    file.write('<sentence ID=' + str(sequence_num) + '>\n')
    
    # body of a sequence, ignore ('<s>', '<s>') and ('</s>', '</s>').
    sequence_stop_val = len(tupls)-1
    for i in range(1, sequence_stop_val): 
        file.write(tupls[i][0] + ', ' + tupls[i][1]+'\n')
    
    # end the sequence. 
    file.write('<EOS>\n') 
    
file.close() 
print('HMM_predictions.txt has been generated.') 

HMM_predictions.txt has been generated.


### Conclusion:   
Using the procedure outlined through steps 2.1 to 2.4, I was able to generate the needed components of an HNN model to generate the viterbi lattice, and ultimitely the tag sequence prediction for a given sentence.

As seen in this notebook, I used a transition probability matrix **A**, an observation matrix containing emission probabilites **B**, a set of N states defined by the **states** variable, a sequence of observations stored in a collection of sequences **test_token_sequences**, and a collection of initial probability distributions **pi**.  

When generating the viterbi lattice, I originally had underflow problems because I was in the multiplicative domain. To circumvent this, I mapped my **initial probability distributions**, **transition probabilities**, and **emission states** to the multiplicitave domain. 

The resulting tags for the test set was written to the 'HMM_predictions.txt' file. 